## FLO Müşteri Segmentasyonu RFM Analizi

In [106]:
import numpy as np
import pandas as pd
import datetime as dt

In [107]:
def data_prep(dataframe):
    dataframe["order_num"] = dataframe["order_num_total_ever_offline"] + dataframe["order_num_total_ever_online"]
    dataframe["value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    
    dataframe["first_order_date"] = pd.to_datetime(dataframe["first_order_date"])
    dataframe["last_order_date"] = pd.to_datetime(dataframe["last_order_date"])
    dataframe["last_order_date_offline"] = pd.to_datetime(dataframe["last_order_date_offline"])
    dataframe["last_order_date_online"] = pd.to_datetime(dataframe["last_order_date_online"])
    
    return dataframe

In [143]:
df_ = pd.read_csv('flo_data_20k.csv')
df = df_.copy()

In [144]:
df = data_prep(df)

In [145]:
df.sample(3)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num,value_total
13522,343de2aa-b006-11e9-9757-000d3a38a36f,Android App,Offline,2019-03-20,2020-06-25,2019-03-21,2020-06-25,2.0,1.0,187.22,285.98,"[KADIN, AKTIFSPOR]",3.0,473.20
5984,095404d4-f133-11e9-9346-000d3a38a36f,Android App,Android App,2019-10-16,2020-06-26,2020-06-26,2019-10-16,1.0,1.0,60.99,244.00,[AKTIFSPOR],2.0,304.99
12936,fe695fe2-1e87-11ea-aa32-000d3a38a36f,Android App,Offline,2019-02-27,2021-02-18,2021-02-10,2021-02-18,2.0,3.0,796.35,1181.44,[AKTIFSPOR],5.0,1977.79


In [146]:
max_date = df["last_order_date"].max()

In [147]:
## Son alışveristen sonraki ikinci günü tespit etmek için 
today_date = max_date + dt.timedelta(2)

In [168]:
"""rfm = df.groupby("master_id").agg({"last_order_date": lambda date: (today_date - date.max()).days,
                                    "order_num": lambda num : num,
                                    "value_total": lambda value: value.sum(),
                                  "master_id": lambda id:id})"""

rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = (today_date - df["last_order_date"]).astype('timedelta64[D]')
rfm["frequency"] = df["order_num"]
rfm["monetary"] = df["value_total"]

In [169]:
rfm.columns

Index(['customer_id', 'recency', 'frequency', 'monetary'], dtype='object')

In [127]:
# rfm.columns = ["recency","frequency","monetary","customer_id"]

In [170]:
rfm

,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0,5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0,2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0,2.0,209.98
...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,331.0,3.0,401.96
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,161.0,2.0,390.47
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,8.0,3.0,632.94
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,108.0,6.0,1009.77


In [171]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

In [172]:
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str)+
                   rfm["frequency_score"].astype(str))

In [173]:
seg_map = {
    r'[1-2][1-2]': "hibernating",
    r"[1-2][3-4]" : "atRisk",
    r"[1-2]5" : "cant_loose",
    r"3[1-2]" : "about_to_sleep",
    r"33":'need_attention',
    r"[3-4][4-5]" : "loyal_customer",
    r"41" : "promising",
    r"51" : "new_customers",
    r"[4-5][2-3]" : "potential_loyalists",
    r"5[4-5]" : "champions"
    }
    
rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex= True)


In [174]:
rfm.columns

Index(['customer_id', 'recency', 'frequency', 'monetary', 'recency_score',
       'frequency_score', 'monetary_score', 'RFM_SCORE', 'segment'],
      dtype='object')

In [175]:
rfm.groupby("segment").agg({
    "recency": lambda avg: avg.mean(),
    "frequency": lambda avg: avg.mean(),
    "monetary": lambda avg: avg.mean(),
})

,recency,frequency,monetary
segment,,,
about_to_sleep,113.785144,2.401473,359.008963
atRisk,241.606835,4.472373,646.610236
cant_loose,235.444167,10.698333,1474.468217
champions,17.106625,8.934265,1406.625109
hibernating,247.949501,2.394007,366.267062
loyal_customer,82.594763,8.374591,1216.818616
need_attention,113.828676,3.727825,562.142965
new_customers,17.917647,2.000000,339.955544
potential_loyalists,37.155888,3.304289,533.184466


In [176]:
rfm[["segment","recency", "frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
about_to_sleep       113.785144  1629   2.401473  1629   359.008963  1629
atRisk               241.606835  3131   4.472373  3131   646.610236  3131
cant_loose           235.444167  1200  10.698333  1200  1474.468217  1200
champions             17.106625  1932   8.934265  1932  1406.625109  1932
hibernating          247.949501  3604   2.394007  3604   366.267062  3604
loyal_customer        82.594763  3361   8.374591  3361  1216.818616  3361
need_attention       113.828676   823   3.727825   823   562.142965   823
new_customers         17.917647   680   2.000000   680   339.955544   680
potential_loyalists   37.155888  2938   3.304289  2938   533.184466  2938
promising             58.921175   647   2.000000   647   335.672705   647

#### 2. RFM analizi yardımı ile 2 case için ilgili profildeki müşterileri bulunuz ve müşteri id'lerini csv ye kaydediniz.
a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçeilmek isteniliyor. Bu müşterilerin sadık  ve kadın kategorisinden alışveriş yapan kişiler olması planlandı. Müşterilerin id numaralarını csv dosyasına yeni_marka_hedef_müşteri_id.cvs olarak kaydediniz.

In [177]:
target = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["customer_id"]

In [179]:
cust_ids = df[(df["master_id"].isin(target)) &(df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
cust_ids.to_csv("new_brand_target_women_id.csv", index=False)

b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşterilerden olan ama uzun süredir alışveriş yapmayan ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv olarak kaydediniz.

In [180]:
target_segments_customer_ids = rfm[rfm["segment"].isin(["cant_loose","hibernating","new_customers"])]["customer_id"]
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) & ((df["interested_in_categories_12"].str.contains("ERKEK"))|(df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
cust_ids.to_csv("discount_target_customer_ids.csv", index=False)